In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset, TensorDataset
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.metrics import average_precision_score, coverage_error, label_ranking_loss, hamming_loss,zero_one_loss
from sklearn.metrics import roc_curve, auc
import warnings
warnings.filterwarnings(action='ignore')

In [7]:
# Loading the training data
train_features = pd.read_csv("../dataset/training_validation.csv")
train_prior_info = pd.read_csv("../dataset/train_prior_information.csv")
train_labels = pd.read_csv("../dataset/training_validation_annotation_encoded.csv")

In [8]:
train_merged_data = pd.concat([train_features, train_prior_info], axis=1)

In [9]:
train_merged_data

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,Prior Probability
0,0.118044,0.105379,-0.062857,0.061380,0.044781,0.078056,0.023524,-0.097733,-0.009063,-0.063286,...,0.044655,-0.053988,-0.062055,-0.044813,-0.016775,-0.019967,0.008345,0.012604,0.047983,0.048937
1,0.107418,0.104035,-0.067825,0.045469,0.026095,0.078852,0.017829,-0.081949,-0.014877,-0.049385,...,0.044975,-0.051977,-0.055258,-0.039324,-0.015723,-0.016805,0.004520,0.012009,0.044528,0.177657
2,0.120856,0.092818,-0.062700,0.039001,0.022148,0.080469,0.011729,-0.073548,-0.011473,-0.050525,...,0.045266,-0.052028,-0.059198,-0.043812,-0.018790,-0.018081,-0.000413,0.016619,0.047241,0.165711
3,0.101818,0.100454,-0.064449,0.040967,0.032523,0.090816,0.020269,-0.076947,-0.016859,-0.045373,...,0.042948,-0.050023,-0.058237,-0.040837,-0.016077,-0.014774,0.000983,0.014381,0.046040,0.199936
4,0.116989,0.091419,-0.065899,0.049676,0.040391,0.078420,0.016239,-0.091415,-0.004523,-0.055382,...,0.045556,-0.053979,-0.065390,-0.046985,-0.017443,-0.020867,0.003168,0.015970,0.050232,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33269,0.098640,0.099309,-0.068083,0.033008,0.029505,0.092849,0.019721,-0.080735,-0.019335,-0.042576,...,0.044555,-0.051535,-0.050830,-0.038268,-0.018233,-0.012395,0.006478,0.011503,0.043658,0.031977
33270,0.105183,0.105032,-0.067741,0.046225,0.023203,0.084734,0.012807,-0.082371,-0.013481,-0.049007,...,0.046226,-0.052876,-0.054073,-0.040365,-0.018346,-0.013928,0.002856,0.013035,0.044841,0.165711
33271,0.108450,0.096900,-0.062981,0.045981,0.025787,0.081892,0.014120,-0.087542,-0.004153,-0.051180,...,0.044700,-0.052309,-0.056901,-0.040256,-0.015901,-0.018089,0.002973,0.013475,0.045291,0.177657
33272,0.103879,0.098663,-0.067586,0.038443,0.027252,0.089530,0.017511,-0.079006,-0.015713,-0.048196,...,0.043720,-0.050924,-0.055212,-0.039763,-0.016813,-0.014098,0.003407,0.013215,0.044934,0.142663


In [10]:
# Load the independence test data
test_features = pd.read_csv("../dataset/independent.csv")
test_prior_info = pd.read_csv("../dataset/test_prior_information.csv")
test_labels = pd.read_csv("../dataset/independent_annotation_encoded.csv")

test_merged_data = pd.concat([test_features, test_prior_info], axis=1)
test_merged_data

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,0
0,0.125636,0.112451,-0.064009,0.078311,0.032094,0.072640,0.015092,-0.095536,-0.000781,-0.068716,...,0.048345,-0.056419,-0.066093,-0.047837,-0.017017,-0.022755,0.003036,0.011886,0.048476,0.016259
1,0.110403,0.088578,-0.068826,0.033890,0.027336,0.074340,0.011718,-0.081953,-0.008908,-0.049788,...,0.045452,-0.052712,-0.060584,-0.042797,-0.016407,-0.020327,0.001603,0.015647,0.047680,0.147510
2,0.090393,0.099999,-0.065848,0.031768,0.025161,0.104376,0.016381,-0.074810,-0.017934,-0.036974,...,0.044323,-0.049816,-0.048414,-0.037420,-0.019882,-0.008316,0.000480,0.013536,0.042691,0.020827
3,0.089444,0.100762,-0.071104,0.026423,0.024162,0.099623,0.018537,-0.075468,-0.023015,-0.036704,...,0.043682,-0.049763,-0.046573,-0.035440,-0.018596,-0.008990,0.004983,0.011808,0.041907,0.053846
4,0.102843,0.110370,-0.072914,0.058539,0.038735,0.089164,0.017744,-0.096406,-0.010803,-0.054107,...,0.047541,-0.055433,-0.055479,-0.042343,-0.018893,-0.014036,0.009185,0.010033,0.045780,0.227724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3692,0.088771,0.092041,-0.062633,0.026916,0.029719,0.103593,0.013505,-0.070907,-0.017409,-0.033421,...,0.044905,-0.050094,-0.048605,-0.038540,-0.021556,-0.008020,-0.000396,0.015200,0.043592,0.065537
3693,0.090001,0.106449,-0.069520,0.044603,0.034526,0.096462,0.018131,-0.085937,-0.016868,-0.045346,...,0.044076,-0.051305,-0.052233,-0.038127,-0.017320,-0.009782,0.005673,0.011523,0.044207,0.121346
3694,0.123868,0.106322,-0.066556,0.052013,0.014493,0.075613,0.010797,-0.080255,-0.009970,-0.060066,...,0.047505,-0.055032,-0.060688,-0.043469,-0.016300,-0.019778,0.003649,0.011605,0.046321,0.149330
3695,0.078793,0.108086,-0.074426,0.036118,0.035230,0.100041,0.017158,-0.087947,-0.021035,-0.033618,...,0.046297,-0.052939,-0.049437,-0.036441,-0.017385,-0.008506,0.008517,0.009348,0.043631,0.165711


In [11]:
# standardization
sc = StandardScaler()
sc.fit(train_merged_data)
train_merged_data = sc.transform(train_merged_data)
test_merged_data = sc.transform(test_merged_data)

In [12]:
class BiLSTMWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BiLSTMWithAttention, self).__init__()
        self.bilstm = nn.LSTM(input_size, hidden_size, bidirectional=True, batch_first=True)
        self.attention = nn.MultiheadAttention(hidden_size * 2, num_heads=9)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        output, _ = self.bilstm(x)
        attention_output, _ = self.attention(output, output, output)
        attention_output = attention_output.permute(1, 0, 2)  # Change the order back
        output = attention_output[:, -1, :]
        output = self.fc(output)  

        return output

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [1]:
from sklearn.metrics import accuracy_score, average_precision_score, coverage_error, hamming_loss
from sklearn.metrics import average_precision_score, coverage_error, label_ranking_loss, hamming_loss,zero_one_loss
# Initialize the list of metrics
acc_exam_list = []
average_precision_list = []
coverage_list = []
one_error_list = []
ranking_loss_list = []
hamming_loss_list = []
kf = KFold(n_splits=10, shuffle=True, random_state=42)
for fold, (train_index, val_index) in enumerate(kf.split(train_merged_data, train_labels)):
    
    X_train, X_val = train_merged_data[train_index], train_merged_data[val_index]
    y_train, y_val = train_labels.iloc[train_index], train_labels.iloc[val_index]

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).to(device)

    # Reshape data into a 3D tensor
    X_train_tensor = X_train_tensor.unsqueeze(1)  # Add a dimension to the second dimension
    X_val_tensor = X_val_tensor.unsqueeze(1)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    model = BiLSTMWithAttention(input_size=X_train.shape[1], hidden_size=126, output_size=train_labels.shape[1]).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    for epoch in tqdm(range(100),desc='Epoch'):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluate on the validation dataset
    model.eval()
    with torch.no_grad():
        outputs = model(X_val_tensor)
        predicted_labels = torch.round(torch.sigmoid(outputs))
        
        
        # calculate each metric
        #      compute intersection and union
        # Convert the label tensor to integer type
        # compute intersection and union
        intersection = torch.logical_and(predicted_labels.bool(), y_val_tensor.bool()).sum(dim=1).float()
        union = torch.logical_or(predicted_labels.bool(), y_val_tensor.bool()).sum(dim=1).float()
        
        # Calculate Acc exam for each fold
        acc_exam_fold = torch.mean(intersection / union).item()
        
        
        acc_exam_list.append(acc_exam_fold)
#         average_precision = average_precision_score(y_val_tensor.cpu().numpy(), outputs.cpu().numpy())
#         coverage = coverage_error(y_val_tensor.cpu().numpy(), outputs.cpu().numpy())
        # Code to calculate other metrics here
        hamming_loss_value = hamming_loss(y_val_tensor.cpu().numpy(), predicted_labels.cpu().numpy())
        average_precision = average_precision_score(y_val_tensor.cpu().numpy(), predicted_labels.cpu().numpy())
        coverage = coverage_error(y_val_tensor.cpu().numpy(), predicted_labels.cpu().numpy())
#         y_val_tensor = torch.round(torch.sigmoid(y_val_tensor))
#         outputs = torch.round(torch.sigmoid(outputs))
        one_error = zero_one_loss(y_val_tensor.cpu().numpy(), predicted_labels.cpu().numpy())
        ranking_loss = label_ranking_loss(y_val_tensor.cpu().numpy(),predicted_labels.cpu().numpy())
        
        
        average_precision_list.append(average_precision)
        coverage_list.append(coverage)
        one_error_list.append(one_error)
        ranking_loss_list.append(ranking_loss)
        hamming_loss_list.append(hamming_loss_value)

# Calculate the mean of the metrics
avg_acc_exam = np.mean(acc_exam_list)
avg_average_precision = np.mean(average_precision_list)
avg_coverage = np.mean(coverage_list)
avg_one_error = np.mean(one_error_list)
avg_ranking_loss = np.mean(ranking_loss_list)
avg_hamming_loss = np.mean(hamming_loss_list)

# print the average metric
print("Average Acc_exam:", avg_acc_exam)
print("Average Average Precision:", avg_average_precision)
print("Average Coverage:", avg_coverage)
print("Average One-error:", avg_one_error)
print("Average Ranking Loss:", avg_ranking_loss)
print("Average Hamming Loss:", avg_hamming_loss)






#Training

In [9]:
train_merged_data_tensor = torch.tensor(train_merged_data, dtype=torch.float32).to(device)
train_labels_tensor = torch.tensor(train_labels.values, dtype=torch.float32).to(device)

In [10]:
train_merged_data_tensor = train_merged_data_tensor.unsqueeze(1)  # Add a dimension to the second dimension

In [11]:
train_dataset = TensorDataset(train_merged_data_tensor, train_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [12]:
model = BiLSTMWithAttention(input_size=3001, hidden_size=126, output_size=9).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

for epoch in tqdm(range(1000),desc='Epoch'):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/1000], Loss: {epoch_loss:.4f}')
# save model
torch.save(model.state_dict(), 'BiLSTM_with_Attention.pth')

Epoch:   0%|                                                                        | 1/1000 [00:04<1:09:51,  4.20s/it]

Epoch [1/1000], Loss: 0.4915


Epoch:   0%|▏                                                                       | 2/1000 [00:07<1:02:30,  3.76s/it]

Epoch [2/1000], Loss: 0.3393


Epoch:   0%|▏                                                                       | 3/1000 [00:11<1:03:45,  3.84s/it]

...

Epoch [998/1000], Loss: 0.1464


Epoch: 100%|███████████████████████████████████████████████████████████████████████▉| 999/1000 [53:46<00:03,  3.21s/it]

Epoch [999/1000], Loss: 0.1464


Epoch: 100%|███████████████████████████████████████████████████████████████████████| 1000/1000 [53:49<00:00,  3.23s/it]

Epoch [1000/1000], Loss: 0.1458


#Testing

In [16]:
# test_features = pd.read_csv("independent.csv")
# test_prior_info = pd.read_csv("test_prior_information.csv")
# test_labels = pd.read_csv("./dataset/independent_annotation_encoded.csv")

In [17]:
# test_merged_data = pd.concat([test_features, test_prior_info], axis=1)
# test_merged_data

In [18]:
test_merged_data_tensor = torch.tensor(test_merged_data, dtype=torch.float32).to(device)
test_labels_tensor = torch.tensor(test_labels.values, dtype=torch.float32).to(device)

In [19]:
test_merged_data_tensor = test_merged_data_tensor.unsqueeze(1) 
# test_labels_tensor = test_labels_tensor.unsqueeze(1)

In [20]:
import torch
import numpy as np
from sklearn.metrics import average_precision_score, coverage_error, hamming_loss, zero_one_loss, label_ranking_loss

# Initialize the model and load parameters
model = BiLSTMWithAttention(input_size=3001, hidden_size=126, output_size=9).to(device)
model.load_state_dict(torch.load('../Model/BiLSTM_with_Attention.pth'))
model.eval()

with torch.no_grad():
    outputs = model(test_merged_data_tensor)
    
    predicted_labels = torch.round(outputs)
    
    # Make sure test_labels_tensor and predicted_labels are PyTorch tensors
    if isinstance(test_labels_tensor, np.ndarray):
        test_labels_tensor = torch.tensor(test_labels_tensor)
    if isinstance(predicted_labels, np.ndarray):
        predicted_labels = torch.tensor(predicted_labels)

    # compute intersection and union
    intersection = torch.logical_and(predicted_labels.bool(), test_labels_tensor.bool()).sum(dim=1).float()
    union = torch.logical_or(predicted_labels.bool(), test_labels_tensor.bool()).sum(dim=1).float()

    # Calculate Acc exam for each fold
    acc_exam = torch.mean(intersection / union).item()

    # Convert the predictions and labels from tensors to NumPy arrays
    test_labels_numpy = test_labels_tensor.cpu().numpy()
    predicted_labels_numpy = predicted_labels.cpu().numpy()
#     outputs_numpy = outputs.cpu().numpy()

    # Remove a single dimension from numpy test labels
    test_labels_numpy = np.squeeze(test_labels_numpy)
    
    # calculate other metrics
    average_precision = average_precision_score(test_labels_numpy, predicted_labels_numpy)
    coverage = coverage_error(test_labels_numpy, predicted_labels_numpy)
    hamming_loss_value = hamming_loss(test_labels_numpy, predicted_labels_numpy)
    one_error = zero_one_loss(test_labels_numpy, predicted_labels_numpy)
    ranking_loss = label_ranking_loss(test_labels_numpy, predicted_labels_numpy)

print("Acc_exam:", acc_exam)
print("Average Precision:", average_precision)
print("Coverage:", coverage)
print("One-error:", one_error)
print("Ranking Loss:", ranking_loss)
print("Hamming Loss:", hamming_loss_value)
